In [1]:
import os

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    os.chdir(os.path.dirname(os.getcwdb()))

from utils.classes import *
from utils.functions import *


# Consideraciones previas

- La "target" del "dataframe" de competición no incluye los precios reales de los diamantes, sino que está escalada de algún modo

- Los precios auténticos se pueden ver en el "dataframe" original sobre los diamantes, que se encuentra aquí: https://www.kaggle.com/datasets/swatikhedekar/price-prediction-of-diamond

- A la hora de subir la predicción con "flask", es importante que sean los verdaderos

- Sin embargo, los índices de los "dataframes" de competición y el de no competición no son comparables

- Por tanto, hay que encontrar la fórmula usada y deshacer el escalado. En este "notebook" se investiga y descubre que se ha usado "np.log()" con la columna "target" del "dataframe" de competición

- Se concluye que habrá que revertir ese escalado con "np.exp()" aplicado en la predicción con tal de ofrecer los precios reales

- Como los precios reales son de 2017, es posible que haya que hacerles alguna otra actualización

# Investigación

- Se demuestra que los diamantes de competición se encuentran en el "dataframe" de no competición, si bien con distintos precios (los reales)

In [2]:
df_no_competition = pd.read_csv(r'data\raw\no_competition.csv')
df_competition_train = pd.read_csv(r'data\raw\train.csv', index_col='id')
df_train = pd.read_csv(r'data\processed\diamonds_training.csv', index_col='id')


In [3]:
# Se reordenan las columnas para que los dos "dataframes" sean idénticos
new_columns = ['carat',	'cut', 'color',	'clarity', 'depth', 'table', 'x', 'y', 'z', 'price']

df_no_competition = df_no_competition.reindex(columns=new_columns)

df_no_competition.head()


,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335


In [4]:
# Por la misma razón, se resetea el índice
df_competition_train = df_competition_train.reset_index(drop=True)

df_competition_train.head()


,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183
2,0.72,Ideal,F,VS2,61.8,59.0,5.71,5.74,3.54,7.983
3,1.08,Very Good,G,SI2,63.2,57.0,6.54,6.50,4.12,8.371
4,0.36,Premium,G,VS1,62.3,59.0,4.50,4.55,2.82,6.588


In [5]:
# Se borran los duplicados
df_no_competition, df_competition_train = df_no_competition.drop_duplicates(), df_competition_train.drop_duplicates()


In [6]:
# En el "dataframe" de competición, se buscan los valores únicos, por ejemplo, de la columna "x", y se detectan varios valores únicos con una sola aparición
df_competition_train['x'].value_counts()


4.37    338
4.34    335
4.38    332
4.33    330
4.32    310
       ... 
3.74      1
9.15      1
9.13      1
9.38      1
9.41      1
Name: x, Length: 546, dtype: int64

In [7]:
# Se aísla uno de ellos
df_competition_train[df_competition_train['x'] == 9.41]


,carat,cut,color,clarity,depth,table,x,y,z,price
40067,3.01,Premium,J,SI2,59.7,58.0,9.41,9.32,5.59,9.837


In [8]:
# Se localiza es emismo valor en el "dataframe" de no competición, y se comprueba que son el mismo diamante
df_no_competition[df_no_competition['x'] == 9.41]


,carat,cut,color,clarity,depth,table,x,y,z,price
27685,3.01,Premium,J,SI2,59.7,58.0,9.41,9.32,5.59,18710


In [9]:
# Se aísla un segundo valor para asegurar
df_competition_train[df_competition_train['x'] == 3.77]


,carat,cut,color,clarity,depth,table,x,y,z,price
10412,0.2,Premium,D,VS2,61.7,60.0,3.77,3.72,2.31,5.905


In [10]:
# Se busca este segundo en el otro dataframe. También existe
df_no_competition[df_no_competition['x'] == 3.77]


,carat,cut,color,clarity,depth,table,x,y,z,price
31601,0.2,Premium,D,VS2,61.7,60.0,3.77,3.72,2.31,367


In [11]:
# Se comprueba que ningún valor del "price" en el "datafame" de competición es superior a 10
any(df_competition_train['price'] > 10)


False

In [12]:
# Asimismo, todos los valores del de competición lo son
all(df_no_competition['price'] > 10)


True

In [13]:
# Se comprueba que todos los valores únicos del "dataframe" de no competición se encuentran en el de competición
for column in df_no_competition.columns[:-1]:
    no_competition_list = df_no_competition[column].unique()
    competition_list = df_competition_train[column].unique()
    print(set(competition_list).issubset(no_competition_list))


True
True
True
True
True
True
True
True
True


# Transformación
- Se descubre que que hay muchos diamantes iguales con precios distintos en el "dataframe" de datos reales

- Se encuentra el escalado de los datos empleado


In [14]:
# El "dataframe" resultante de unir ambos es mayor que el de competición. "price_x" es el precio real, y "price_y" es el de la competición
df_merged = pd.merge(df_no_competition, df_competition_train, on=['carat',	'cut', 'color',	'clarity', 'depth', 'table', 'x', 'y', 'z'], how='right', indicator=True)

df_merged


,carat,cut,color,clarity,depth,table,x,y,z,price_x,price_y,_merge
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,574,6.353,both
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9735,9.183,both
2,0.72,Ideal,F,VS2,61.8,59.0,5.71,5.74,3.54,2931,7.983,both
3,1.08,Very Good,G,SI2,63.2,57.0,6.54,6.50,4.12,4318,8.371,both
4,0.36,Premium,G,VS1,62.3,59.0,4.50,4.55,2.82,726,6.588,both
...,...,...,...,...,...,...,...,...,...,...,...,...
40672,0.42,Premium,D,SI2,62.1,59.0,4.78,4.82,2.98,700,6.551,both
40673,0.53,Premium,G,VS2,62.0,58.0,5.21,5.18,3.22,1607,7.382,both
40674,0.80,Good,G,SI2,62.8,58.0,5.86,5.90,3.69,2363,7.768,both
40675,1.01,Very Good,F,VS2,61.5,57.0,6.40,6.48,3.96,6159,8.726,both


In [15]:
# Ello se debe a que hay muchas filas del "dataframe" de no competición que tienen precios distintos para los mismos diamantes
print(len(df_no_competition.drop(columns='price')))
print(len(df_no_competition.drop(columns='price').drop_duplicates()))


53794
53595


In [17]:
# Se prueba con aplicar el logaritmo a la columna de precio real ("price_x"). El resultado es casi idéntico al de la columna del precio de competición ("price_y")
df_merged['log_price_x'] = np.log(df_merged['price_x'])

df_merged.head()
